origin: http://pytorch.org/tutorials/beginner/nlp/pytorch_tutorial.html  
translator: Hongpu Liu

In [1]:
%matplotlib inline
from __future__  import print_function, division

# 词嵌入：对词汇语义进行编码
词嵌入是一个字典中每一个词对应的稠密实数向量。在自然语言处理中，几乎大部分情况都会选用词作为特征。但是如何向计算机表达一个词？你可以存储词的ASCII表示，但是这样只能知道这是那个词，而不知道它的含义（你可能可以通过词性词缀，或者从首字母大写推导一些属性，但是这样的信息并不充分）。除此之外，是否能够把这些表达进行组合？神经网络通常需要一个稠密的输出，例如输入为$|V|$维（$V$是词汇表）的神经网络，其输出只有很小的维度（例如，我们的预测只包含少量的标签）。我们如何从非常高维的空间降至低维空间？

用独热编码来代替ASCII表示如何？这种形式的编码把一个词$w$表示为：

$$\overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements}$$

其中$1$的位置对于$w$是唯一的，其余的位置都是0。任意其他的词都有不同的$1$的位置。

除了向量过长，这种表示方法还有很多缺点。这种表示把所有的词看作独立的实体，词与词之间没有任何的关系。而我们真正想要的是词与词之间的某种相似度。我们来看下面的例子。

假设我们要构造一个语言模型，在数据集中有以下句子：
- The mathematician ran to the store.
- The physicist ran to the store.
- The mathematician solved the open problem.

现在假设我们得到了之前未曾在训练集中见过的新句子：
- The physicist solved the open problem.

我们的语言模型可能对这个句子进行正确的处理，但是如果使用下面的事实表现会更好：
- 我们看到 mathematicians 和 physicist 在句子中担任了同样的角色，因此他们之间存在某种语义关系。
- 我们看到 mathematician 与新的未曾见过的句子中的 physicist 担任同样的语法角色。

然后我们推断 physicist 对于新的未曾见过的句子非常适合。这就是之前说到的某种相似度：语法相似度，而不是简单的正交表示。通过连接见过的句子与未曾见过的句子来解决语言数据的稀疏性是一项技术。这个例子依赖于一个基本的语言学假设：能出现在相似上下文中的词与他们之间的语义相关。这叫做[分布假设](https://en.wikipedia.org/wiki/Distributional_semantics)。

## 1. 获得稠密词嵌入
如何解决这个问题？如何把词的语义相似度进行编码？也许我们可以设计出一些语义属性。例如，由于我们看到 mathematicians 和 physicists 都可以跑，也许我们可以给这两个词的“能跑”属性较高的分数。再设计其他的一些属性，然后为这些词在不同的属性上进行评分。

每个属性是一个维度，然后我们可以把每个词用向量来表示：

$$q_\text{mathematician}=\left[ \overbrace{2.3}^\text{can run}, \overbrace {9.4}^\text{likes coffee}, \overbrace {-5.5}^\text{majored in Physics}, \dots \right]$$

$$q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run}, \overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right]$$

然后我们就可以获得词与词之间的相似度：

$$\text{Similarity}(\text{physicist}, \text{mathematician}) = q_\text{physicist} \cdot q_\text{mathematician}$$

更常用的方式是归一化向量的长度：

$$\text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}} {\| q_\text{\physicist} \| \| q_\text{mathematician} \|} = \cos (\phi)$$

其中$\phi$是两个向量的夹角。在这种相似性度量下，非常近似的词（词的嵌入点具有相同的方向）相似度为$1$。完全不相似的词相似度为$-1$。

可以把本节开始的独热向量看作这种新的向量形式的特例：所有词之间的相似度都是$0$，每个词都被赋予了一个唯一的语义属性。这些新的向量是**稠密**的，也就是说他们的元素几乎都是非0的。

这种新的向量定义方式最大的问题在于：我们可能会想到成千上万的用于确定相似度的语义属性。如何为每一个词设置不同属性的值？深度学习的中心思想就是利用神经网络学习特征的表示，而不是手工设计。正因如此，何不将词嵌入作为模型的参数，在训练的过程中更新它。这就是要做的。

我们的网络将会学习一些**潜在的语义属性**。要注意的是词嵌入很可能是无法解释的。在我们手工设计的词嵌入中， mathematician 和 physicist 在“likes coffee”属性上都有很高的分数。如果我们用神经网络来学习词嵌入， mathematician 和 physicist可能都在第二个维度上有很高的分数，但是我们并不知道这意味着什么。我们能够知道在这两个词在某些潜在的语义维度上相似，但是很可能无法解释其具体含义。

总的来说，**词嵌入是一个词*语义*的表示，高效的编码语义信息与处理的任务相关。**除此之外，词性标记、语法树等都可以进行嵌入。特征嵌入是这个领域的研究中心。

## 2. 用PyTorch实现词嵌入
在我们开始进行自然语言处理任务之前，首先看一下在PyTorch和深度学习中如何使用词嵌入。类似于之前定义的用独热向量表示词的方法，在词嵌入中，每个词被赋予一个唯一的索引。该索引将被作为查表的关键字。也就是说，**嵌入**存储于一个大小为$|V| \times D$的矩阵中，其中$D$是嵌入空间的维度，若一个词的索引为$i$，那么它的嵌入存储于该矩阵的第$i$行。在本教程中，从词到索引的映射存储于一个叫做**word_to_ix**的词典中。

PyTorch中，**torch.nn.Embedding**用于实现嵌入功能，它需要两个参数：词典的大小和嵌入空间的维度。

表的索引必须是整数，因此我们要用**torch.LongTensor**作为索引的类型。

In [2]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5) # 词典共有2个词，嵌入空间为5维
lookup_tensor = torch.LongTensor([word_to_ix["hello"]])
hello_embed = embeds(autograd.Variable(lookup_tensor))
print(hello_embed)

Variable containing:
 0.6614  0.2669  0.0617  0.6213 -0.4519
[torch.FloatTensor of size 1x5]



## 3. 例子：N元语言模型
首先回顾一下N元语言模型，给定一个词序列$w$，我们可以计算：

$$P(w_i | w_{i-1}, w_{i-2}, \dots, w_{i-n+1} )$$
其中$w_i$是序列中的第i个词。

在下面的例子中，我们将计算一些训练样本的损失函数，并利用反向传播更新参数。

In [4]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

# 本例使用莎士比亚14行诗的第二首
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# 我们本该先做分词，但是这次将忽略这一过程

# 构造一个元组构成的列表，每个元组均为： ([word_i-2, word_i-1], target word)
trigrams = [ ([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2]) 
            for i in range(len(test_sentence) - 2)]
# 打印前三个成员，这样我们可以看其结构
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

class NGramLanguageModeler(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(20):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:
        
        # 步骤1：准备要传给模型的输入，将词转换为整数索引并封装到变量中
        context_idxs = [word_to_ix[w] for w in context]
        context_var = autograd.Variable(torch.LongTensor(context_idxs))
        
        # 步骤2：由于torch会累积梯度，因此开始传递样本之前，首先把旧样本的梯度清空
        model.zero_grad()
        
        # 步骤3：运行前馈过程，获得下一个词的对数概率分布
        log_probs = model(context_var)
        
        # 步骤4：计算损失函数，注意Torch需要将目标单词封装到变量中
        loss = loss_function(log_probs, autograd.Variable(
            torch.LongTensor([word_to_ix[target]])))
        
        # 步骤5： 做反向传播计算梯度，并更新权重
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data
    losses.append(total_loss)

print(losses)

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
[
 521.6290
[torch.FloatTensor of size 1]
, 
 519.0342
[torch.FloatTensor of size 1]
, 
 516.4572
[torch.FloatTensor of size 1]
, 
 513.8986
[torch.FloatTensor of size 1]
, 
 511.3573
[torch.FloatTensor of size 1]
, 
 508.8298
[torch.FloatTensor of size 1]
, 
 506.3180
[torch.FloatTensor of size 1]
, 
 503.8196
[torch.FloatTensor of size 1]
, 
 501.3337
[torch.FloatTensor of size 1]
, 
 498.8603
[torch.FloatTensor of size 1]
, 
 496.3966
[torch.FloatTensor of size 1]
, 
 493.9445
[torch.FloatTensor of size 1]
, 
 491.5018
[torch.FloatTensor of size 1]
, 
 489.0665
[torch.FloatTensor of size 1]
, 
 486.6394
[torch.FloatTensor of size 1]
, 
 484.2194
[torch.FloatTensor of size 1]
, 
 481.8055
[torch.FloatTensor of size 1]
, 
 479.3979
[torch.FloatTensor of size 1]
, 
 476.9951
[torch.FloatTensor of size 1]
, 
 474.5963
[torch.FloatTensor of size 1]
]


## 4. 练习：计算词嵌入——连续词袋
连续词袋模型（CBOW）在深度NLP中经常被用到。该模型试图在给定之前若干个词和之后若干个词时来预测目标单词。CBOW在语言模型中非常独特，它既不是序列也不需要输出概率分布。通常，CBOW用来快速训练词嵌入，然后训练出来的词嵌入可以用来初始化更复杂模型的嵌入层。通常这被称为**预训练嵌入**。它总会能让性能提高几个百分点。

CBOW模型如下所示。给定目标词$w_i$和在两边长度为$N$的上下文窗口：$w_{i-1},\dots,w_{i-N}$和$w_{i+1},\dots,w_{i+N}$。记所有上下文的单词集合为$C$，CBOW试图最小化：

$$ -\log p(w_i | C) = -\log \text{Softmax}(A(\sum_{w \in C} q_w) + b)$$
其中$q_w$是词$w$的嵌入。

用PyTorch实现此模型。一些提示：
- 考虑一下都需要定义哪些参数。
- 确保知道每一步运算的形状是什么样的，如果需要改变形状可以使用**.view()**。

In [5]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
# print(data[:5])


class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        out = torch.sum(embeds, 0).view((1, -1))
        out = self.linear(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs
        

# create your model and train.  here are some functions to help you make
# the data ready for use by your module


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)


#make_context_vector(data[0][0], word_to_ix)  # example

losses = []
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in data:
        
        context_var = make_context_vector(context, word_to_ix)
        
        model.zero_grad()
        
        log_probs = model(context_var)
        
        loss = loss_function(log_probs, autograd.Variable(
            torch.LongTensor([word_to_ix[target]])))
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data
    losses.append(total_loss)
    
print(losses)

[
 238.7214
[torch.FloatTensor of size 1]
, 
 236.8414
[torch.FloatTensor of size 1]
, 
 234.9862
[torch.FloatTensor of size 1]
, 
 233.1553
[torch.FloatTensor of size 1]
, 
 231.3478
[torch.FloatTensor of size 1]
, 
 229.5633
[torch.FloatTensor of size 1]
, 
 227.8012
[torch.FloatTensor of size 1]
, 
 226.0611
[torch.FloatTensor of size 1]
, 
 224.3423
[torch.FloatTensor of size 1]
, 
 222.6446
[torch.FloatTensor of size 1]
]
